In [3]:
from capo.analysis.utils import get_results
import pandas as pd
import os

os.chdir("../")

In [4]:
models = ["llama", "qwen", "mistral"]
datasets = ["sst-5", "agnews", "copa", "gsm8k"]
optims = ["CAPO", "EvoPromptGA", "PromptWizard", "OPRO"]

ablations = [
    "CAPO_no_lp",
    "CAPO_generic_init",
    "CAPO_no_racing",
    "CAPO_no_shuffling",
    "CAPO_zero_shot",
    "EvoPromptGA_generic_init",
    "EvoPromptGA_TD",
    "CAPO_gamma_0.1",
    "CAPO_gamma_0.01",
    "CAPO_gamma_0.02",
    "CAPO_gamma_0.05",
    "CAPO_ncrossovers_4",
    "CAPO_ncrossovers_7",
    "CAPO_ncrossovers_10",
    "CAPO_pop_8",
    "CAPO_pop_10",
    "CAPO_pop_12",
]

In [5]:
dfs = []

for model in models:
    for dataset in datasets:
        for optim in optims:
            df = get_results(
                dataset,
                model,
                optim,
            )
            dfs.append(df.assign(model=model, dataset=dataset, optim=optim))

for ablation in ablations:
    for dataset in ["agnews", "gsm8k"]:
        df = get_results(dataset, "llama", ablation)
        dfs.append(df.assign(model=model, dataset=dataset, optim=ablation))

df = pd.concat(dfs, ignore_index=True)

Failed to load agnews for CAPO_no_shuffling: No objects to concatenate
Failed to load gsm8k for CAPO_no_shuffling: No objects to concatenate


C:\Users\tzehl\AppData\Local\Temp\ipykernel_12952\3970149590.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(dfs, ignore_index=True)


In [6]:
# find first steps
df_min = df.groupby(["model", "dataset", "optim", "seed"]).first()["timestamp"]
df_min = pd.to_datetime(df_min)
df_max = df.groupby(["model", "dataset", "optim", "seed"]).last()["timestamp"]
df_max = pd.to_datetime(df_max)

In [7]:
df = df_max - df_min

In [9]:
df.sum()

Timedelta('13 days 04:09:55.649369')